<a href="https://colab.research.google.com/github/pietrzakkuba/computer-assisted-translation-labs/blob/main/lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Komputerowe wspomaganie tłumaczenia

# Zajęcia 3 - terminologia

Na dzisiejszych zajęciach zajmiemy się bliżej słownikami używanymi do wspomagania tłumaczenia. Oczywiście na rynku dostępnych jest bardzo wiele słowników w formacie elektronicznym. Wiele z nich jest gotowych do użycia w SDL Trados, memoQ i innych narzędziach CAT. Zawierają one setki tysięcy lub miliony haseł i oferują natychmiastową pomoc tłumaczowi.

Problem jednak w tym, iż często nie zawierają odpowiedniej terminologii specjalistycznej - używanej przez klienta zamawiającego tłumaczenie. Terminy specjalistyczne są bardzo częste w tekstach tłumaczonych ze względu na następujące zjawiska:
- Teksty o tematyce ogólnej są tłumaczone dość rzadko (nikt nie tłumaczy pocztówek z pozdrowieniami z wakacji...)
- Te same słowa mogą mieć zarówno znaczenie ogólne, jak i bardzo specjalistyczne (np. "dziedziczenie" w kontekście prawnym lub informatycznym)
- Klient używa nazw lub słów wymyślonych przez siebie, np. na potrzeby marketingowe.

Nietrywialnymi zadaniami stają się: odnalezienie terminu specjalistycznego w tekście źródłowym oraz podanie prawidłowego tłumaczenia tego terminu na język docelowy

Brzmi prosto? Spróbujmy wykonać ręcznie tę drugą operację.

### Ćwiczenie 1: Podaj tłumaczenie terminu "prowadnice szaf metalowych" na język angielski. Opisz, z jakich narzędzi skorzystałaś/eś.

Odpowiedź:

```
metal drawer slides / slides for metal drawer
co zrobiłem:
wpisałem hasło w google po polsku, i szukałem na stronach sklepów budowlanych tych prowadnic,
aż znalazłem na którejś stronie prowadnicę wraz z jej kodem produktu
https://ekorex.pl/pl/p/PROWADNICA-MODERN-SLIDE-PW-350MM-3D/33459?gclid=CjwKCAjwxuuCBhATEiwAIIIz0b-EEAey1oQiEOi2fbxjT7hINjxAcouqxQbTatMuNwhdJXjPa5zMOBoCtYYQAvD_BwE. 
Kod (0SHX-18 3D) wkleiłem do google, znalazłem opis produktu po pl, eng i ru w pdfie
https://www.gtv.com.pl/images/products/files/298216_Karta%20katalogowa%20WWW%20PL.pdf
tam było to hasło drawer slides
chciałem się upewnić, że to na pewno to, więc wpisałem drawer slides do google i faktycznie jest to to samo :)

```



W dalszych ćwiczeniach skupimy się jednak na odszukaniu terminu specjalistycznego w tekście. W tym celu będą potrzebne dwie operacje:
1. Przygotowanie słownika specjalistycznego.
2. Detekcja terminologii przy użyciu przygotowanego słownika specjalistycznego.

Zajmijmy się najpierw krokiem nr 2 (gdyż jest prostszy). Rozważmy następujący tekst:

In [7]:
text = " For all Java programmers:"
text += " This section explains how to compile and run a Swing application from the command line."
text += " For information on compiling and running a Swing application using NetBeans IDE,"
text += " see Running Tutorial Examples in NetBeans IDE. The compilation instructions work for all Swing programs"
text += " — applets, as well as applications. Here are the steps you need to follow:"
text += " Install the latest release of the Java SE platform, if you haven't already done so."
text += " Create a program that uses Swing components. Compile the program. Run the program."

Załóżmy, że posiadamy następujący słownik:

In [15]:
dictionary = ['program', 'application', 'applet', 'compile']

### Ćwiczenie 2: Napisz program, który wypisze pozycje wszystkich wystąpień poszczególnych terminów specjalistycznych. Dla każdego terminu należy wypisać listę par (pozycja_startowa, pozycja końcowa).

In [49]:
import re

def terminology_lookup():
    text_lower = text.lower()
    pairs = {}
    for pos in dictionary:
        pairs[pos] = [(occurence.start(), occurence.end()) for occurence in list(re.finditer(pos, text_lower))]
    return pairs
terminology_lookup()

{'applet': [(302, 308)],
 'application': [(80, 91), (164, 175), (322, 333)],
 'compile': [(56, 63), (504, 511)],
 'program': [(14, 21), (291, 298), (468, 475), (516, 523), (533, 540)]}

Zwykłe wyszukiwanie w tekście ma pewne wady. Na przykład, gdy szukaliśmy słowa "program", złapaliśmy przypadkiem słowo "programmer". Złapaliśmy także słowo "programs", co jest poprawne, ale niepoprawnie podaliśmy jego pozycję w tekście.

Żeby poradzić sobie z tymi problemami, musimy wykorzystać techniki przetwarzania języka naturalnego. Wypróbujmy pakiet spaCy:

`pip3 install spacy`

oraz

`python3 -m spacy download en_core_web_sm`

In [33]:
import spacy
nlp = spacy.load("en_core_web_sm")

doc = nlp(text)

for token in doc:
    print(token.lemma_)

 
for
all
Java
programmer
:
this
section
explain
how
to
compile
and
run
a
swing
application
from
the
command
line
.
for
information
on
compile
and
run
a
Swing
application
use
NetBeans
IDE
,
see
Running
Tutorial
Examples
in
NetBeans
IDE
.
the
compilation
instruction
work
for
all
swing
program
—
applet
,
as
well
as
application
.
here
be
the
step
-PRON-
need
to
follow
:
install
the
late
release
of
the
Java
SE
platform
,
if
-PRON-
have
not
already
do
so
.
create
a
program
that
use
swing
component
.
compile
the
program
.
run
the
program
.


Sukces! Nastąpił podział tekstu na słowa (tokenizacja) oraz sprowadzenie do formy podstawowej każdego słowa (lematyzacja).

### Ćwiczenie 3: Zmodyfikuj program z ćwiczenia 2 tak, aby zwracał również odmienione słowa. Na przykład, dla słowa "program" powinien znaleźć również "programs", ustawiając pozycje w tekście odpowiednio dla słowa "programs". Wykorzystaj właściwość idx tokenu.

In [54]:
def terminology_lookup():
    pairs = {}
    for pos in dictionary:
        pairs[pos] = [(token.idx, token.idx + len(token)) for token in doc if token.lemma_ == pos]
    return pairs
terminology_lookup()

{'applet': [(302, 309)],
 'application': [(80, 91), (164, 175), (322, 334)],
 'compile': [(56, 63), (134, 143), (504, 511)],
 'program': [(291, 299), (468, 475), (516, 523), (533, 540)]}

Teraz czas zająć się problemem przygotowania słownika specjalistycznego. W tym celu napiszemy nasz własny ekstraktor terminologii. Wejściem do ekstraktora będzie tekst zawierający specjalistyczną terminologię. Wyjściem - lista terminów.

Przyjmijmy następujące podejście - terminami specjalistycznymi będą najcześćiej występujące rzeczowniki w tekście. Wykonajmy krok pierwszy:

### Ćwiczenie 4: Wypisz wszystkie rzeczowniki z tekstu. Wykorzystaj możliwości spaCy.

In [62]:
def get_nouns(text):
    return [token for token in nlp(text) if token.pos_ == 'NOUN']
get_nouns(text)

[programmers,
 section,
 Swing,
 application,
 command,
 line,
 information,
 application,
 compilation,
 instructions,
 Swing,
 programs,
 applets,
 applications,
 steps,
 release,
 platform,
 program,
 Swing,
 components,
 program,
 program]

Teraz czas na podliczenie wystąpień poszczególnych rzeczowników. Uwaga - różne formy tego samego słowa zliczamy razem jako wystąpienia tego słowa (np. "program" i "programs"). Najwygodniejszą metodą podliczania jest zastosowanie tzw. tally (po polsku "zestawienie"). Jest to słownik, którego kluczem jest słowo w formie podstawowej, a wartością liczba wystąpień tego słowa, wliczając słowa odmienione. Przykład gotowego tally:

In [63]:
tally = {"program" : 4, "component" : 1}

### Ćwiczenie 5: Napisz program do ekstrakcji terminologii z tekstu według powyższych wytycznych.

In [69]:
def extract_terms(text):
    tally = {}
    for token in nlp(text):
        if token.pos_ == 'NOUN':
            if (token.lemma_ not in tally):
                tally[token.lemma_] = 1
            else:
                tally[token.lemma_] += 1
    return tally        
extract_terms(text)

{'applet': 1,
 'application': 3,
 'command': 1,
 'compilation': 1,
 'component': 1,
 'information': 1,
 'instruction': 1,
 'line': 1,
 'platform': 1,
 'program': 4,
 'programmer': 1,
 'release': 1,
 'section': 1,
 'step': 1,
 'swing': 3}

### Ćwiczenie 6: Rozszerz powyższy program o ekstrację czasowników i przymiotników.

In [71]:
def extract_terms(text):
    tally = {}
    for token in nlp(text):
        if token.pos_ == 'NOUN' or token.pos_ == 'VERB' or token.pos_ == 'ADJ':
            if (token.lemma_ not in tally):
                tally[token.lemma_] = 1
            else:
                tally[token.lemma_] += 1
    return tally  
extract_terms(text)

{'applet': 1,
 'application': 3,
 'command': 1,
 'compilation': 1,
 'compile': 2,
 'component': 1,
 'create': 1,
 'do': 1,
 'explain': 1,
 'follow': 1,
 'information': 1,
 'install': 1,
 'instruction': 1,
 'late': 1,
 'line': 1,
 'need': 1,
 'platform': 1,
 'program': 4,
 'programmer': 1,
 'release': 1,
 'run': 3,
 'section': 1,
 'see': 1,
 'step': 1,
 'swing': 3,
 'use': 2,
 'work': 1}